In [ ]:
import os
import sys
sys.path.extend(["./", "../"])

import yaml
import pickle
import argparse

import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

from src.utils.graph_ops import load_graph

In [ ]:
ConfigPATH = '../config/run.yaml'
with open(ConfigPATH, 'r') as f:
    model_config = yaml.safe_load(f)
config = argparse.Namespace(**model_config)

In [ ]:
edge_df = pd.read_csv(f'{config.Feature_PATH}/reference/cleaned_edge_energy_df_related_nucleosome_v021226.csv')
edge_df

In [ ]:
edge_df.columns

In [ ]:
node_w_feature_df = pd.read_csv(f'{config.Feature_PATH}/reference/merged_features_v021226.csv')
# useable_nodes = list(node_w_feature_df.node_id)

In [ ]:
usable_nodes = list(node_w_feature_df.node_id)

In [ ]:
edge_df = edge_df[edge_df['nodeid_1'].isin(usable_nodes) & edge_df['nodeid_2'].isin(usable_nodes)]

In [ ]:
edge_df

# Graph Generation

In [ ]:
G = nx.from_pandas_edgelist(
    edge_df, 
    source='nodeid_1', 
    target='nodeid_2',
    edge_attr=['cleaned_total_energy'] # if you need to save edge weight
)
print(G)

In [ ]:
GraphSAVE = os.path.join(config.Feature_PATH, 'reference/weighted_inter-chain_grpah_nucleosome_v021226.pkl')

with open(GraphSAVE, 'wb') as f:
    pickle.dump(G, f)

## [Action Required: run attProcessing] Merge Topological Feature into Graph

In [ ]:
attr_mapping = {
    'graph_with_betweenness.pkl': 'betweenness',
    'graph_with_pagerank.pkl': 'pagerank',
    'graph_with_closeness.pkl': 'closeness',
    'graph_with_degree.pkl': 'degree',
    'graph_with_shortest_path_length_per_node.pkl': 'shortest_path'
}

In [ ]:
attSAVEPATH = os.path.join(config.Feature_PATH, 'graph_att')

for file_name in os.listdir(attSAVEPATH):
    if not file_name.endswith('.pkl'): continue
    
    with open(os.path.join(attSAVEPATH, file_name), 'rb') as f:
        temp_G = pickle.load(f)
        
        first_node = next(iter(temp_G.nodes()))
        actual_attr_names = temp_G.nodes[first_node].keys()
        
        for attr_name in actual_attr_names:
            attr_values = nx.get_node_attributes(temp_G, attr_name)
            nx.set_node_attributes(G, attr_values, name=attr_mapping[file_name])

In [ ]:
first_node = next(iter(G.nodes()))
actual_attr_names = G.nodes[first_node].keys()
print(actual_attr_names)
print(G)

In [ ]:
FinalattSAVEPATH = os.path.join(config.Feature_PATH, 'final_graph_with_topological_v021226.pkl')
with open(FinalattSAVEPATH, 'wb') as f:
    pickle.dump(G, f)

# Graph EDA

In [ ]:
FinalattSAVEPATH = os.path.join(config.Feature_PATH, 'final_graph_with_topological_v021226.pkl')
G = load_graph(FinalattSAVEPATH)

In [ ]:
connected_components = list(nx.connected_components(G))
cc_sizes = [len(cc) for cc in connected_components]
cc_sizes.sort(reverse=True)
print(len(cc_sizes))

plt.figure(figsize=(10, 4))
plt.hist(cc_sizes, bins=50, edgecolor='black', color='skyblue', alpha=0.7)

plt.xlabel("Connected Component Size", fontsize=8)
plt.ylabel("Frequency (Log Scale)", fontsize=8)
plt.yscale('log')
plt.title("Connected Component Size Distribution in Inter-chain Graph", fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.hist(cc_sizes[26:], bins=50, edgecolor='black', color='skyblue', alpha=0.7)

plt.xlabel("Connected Component Size", fontsize=8)
plt.ylabel("Frequency (Log Scale)", fontsize=8)
plt.yscale('log')
plt.title("Connected Component Size Distribution in Inter-chain Graph", fontsize=10)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()